In [1]:
from azure.storage.blob import BlockBlobService

In [2]:
account_name = 'pdfstorage2'
account_key='js0G39O0qJeihxITkn/JqB2cuWqqy3CykcjWIWfc0ID6a20/ydeWwZc6FT0koQhCJBrnl9YqKTYSzie9Vhph0Q=='
# container_name = "pdfcontainer2"

In [3]:
# Create the BlockBlockService that is used to call the Blob service for the storage account
block_blob_service = BlockBlobService(account_name=account_name, account_key=account_key)

In [4]:
print("\nList blobs in the container")
container_name= "pdfcontainer2"
generator = block_blob_service.list_blobs(container_name)
for blob in generator:
    print("\t Blob name: " + blob.name)


List blobs in the container
	 Blob name: report1.pdf
	 Blob name: report1.pdf.tsv
	 Blob name: report1.pdf.txt
	 Blob name: report2.pdf
	 Blob name: report3.pdf
	 Blob name: report4.pdf
	 Blob name: report5.pdf
	 Blob name: report6.pdf
	 Blob name: report7.pdf


In [6]:
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice, TagExtractor
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.cmapdb import CMapDB
from pdfminer.layout import LAParams
from pdfminer.image import ImageWriter 

In [7]:
# defining function download_file
def download_file(download_url):
    response = urllib2.urlopen(download_url)
    #if file exists remove it
    try:
        os.remove("document.pdf")
    except OSError:
        pass
    file = open("document.pdf", 'wb')
    file.write(response.read())
    file.close()
    print("**** Download Completed ***********")

In [8]:
# defining function pdf2txt
def pdf2txt(argv):
    import getopt
    def usage():
        print ('usage: %s [-d] [-p pagenos] [-m maxpages] [-P password] [-o output]'
               ' [-C] [-n] [-A] [-V] [-M char_margin] [-L line_margin] [-W word_margin]'
               ' [-F boxes_flow] [-Y layout_mode] [-O output_dir] [-R rotation]'
               ' [-t text|html|xml|tag] [-c codec] [-s scale]'
               ' file ...' % argv[0])
        return 100
    try:
        (opts, args) = getopt.getopt(argv[1:], 'dp:m:P:o:CnAVM:L:W:F:Y:O:R:t:c:s:')
    except getopt.GetoptError:
        return usage()
    if not args: return usage()
    # debug option
    debug = 0
    # input option
    password = ''
    pagenos = set()
    maxpages = 0
    # output option
    outfile = None
    outtype = None
    imagewriter = None
    rotation = 0
    layoutmode = 'normal'
    codec = 'utf-8'
    pageno = 1
    scale = 1
    caching = True
    showpageno = True
    laparams = LAParams()
    for (k, v) in opts:
        #print 'k,v=',k,v
        if k == '-d': debug += 1
        elif k == '-p': pagenos.update( int(x)-1 for x in v.split(',') )
        elif k == '-m': maxpages = int(v)
        elif k == '-P': password = v
        elif k == '-o': outfile = v
        elif k == '-C': caching = False
        elif k == '-n': laparams = None
        elif k == '-A': laparams.all_texts = True
        elif k == '-V': laparams.detect_vertical = True
        elif k == '-M': laparams.char_margin = float(v)
        elif k == '-L': laparams.line_margin = float(v)
        elif k == '-W': laparams.word_margin = float(v)
        elif k == '-F': laparams.boxes_flow = float(v)
        elif k == '-Y': layoutmode = v
        elif k == '-O': imagewriter = ImageWriter(v)
        elif k == '-R': rotation = int(v)
        elif k == '-t': outtype = v
        elif k == '-c': codec = v
        elif k == '-s': scale = float(v)
    #
    PDFDocument.debug = debug
    PDFParser.debug = debug
    CMapDB.debug = debug
    PDFResourceManager.debug = debug
    PDFPageInterpreter.debug = debug
    PDFDevice.debug = debug
    #
    rsrcmgr = PDFResourceManager(caching=caching)
    if not outtype:
        outtype = 'text'
        if outfile:
            if outfile.endswith('.htm') or outfile.endswith('.html'):
                outtype = 'html'
            elif outfile.endswith('.xml'):
                outtype = 'xml'
            elif outfile.endswith('.tag'):
                outtype = 'tag'
    if outfile:
        outfp = file(outfile, 'w')
    else:
        outfp = sys.stdout
    if outtype == 'text':
        device = TextConverter(rsrcmgr, outfp, codec=codec, laparams=laparams,
                               imagewriter=imagewriter)
    elif outtype == 'xml':
        device = XMLConverter(rsrcmgr, outfp, codec=codec, laparams=laparams,
                              imagewriter=imagewriter)
    elif outtype == 'html':
        device = HTMLConverter(rsrcmgr, outfp, codec=codec, scale=scale,
                               layoutmode=layoutmode, laparams=laparams,
                               imagewriter=imagewriter)
    elif outtype == 'tag':
        device = TagExtractor(rsrcmgr, outfp, codec=codec)
    else:
        return usage()
    for fname in args:
        fp = file(fname, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        for page in PDFPage.get_pages(fp, pagenos,
                                      maxpages=maxpages, password=password,
                                      caching=caching, check_extractable=True):
            page.rotate = (page.rotate+rotation) % 360
            interpreter.process_page(page)
        fp.close()
    device.close()
    outfp.close()
    return

In [9]:
import pandas as pd 
import os
import glob
import requests
import sys
import urllib2

In [19]:
# if ((len(account_key) > 70) & (container_name != '') & (account_name != '')):
#     doption = 'fromblob'
if ((url != '')):
    doption = 'fromurl'
else:
    doption = ''

In [53]:
# if doption == 'fromurl':
#     url = url
#     download_file(url)
#     #where did it save it?
#     #check the file size
#     pdfname = './document.pdf' #hardcoded the downloaded pdf name to document.pdf
#     sizeofdownload = os.path.getsize(pdfname)  
#     if sizeofdownload == 0:
#         print 'file not downloaded'
#         sys.exit("file not downloaded")
#     else:
#         print 'file size of downloaded pdf = ',sizeofdownload

#     #calling function pdf2txt defined above
#     print "calling function pdf2txt \n"
#     #
#     print 'pdfname = ',pdfname
#     print os.path.abspath(pdfname)
#     textfile = pdfname + '.txt' #local file
#     print 'textfile=',textfile
#     print os.path.abspath(textfile)

#     pdf2txt(["firstarg","-o",textfile,pdfname])
#     print 'pdf converted to text and saved as text in',textfile
#     print os.path.getsize(textfile)
# pdfname='report1.pdf'

#print  pdfname + ' | ' + container + ' | ' + account + ' | ' + acckey + '\n' 
# print "doption is either 'fromblob' or from 'fromurl'"
# print 'download option form user = ',doption

if doption == 'fromblob':
    print 'user BYOB'
    pdflocal = './' + pdfname #local file
    #print 'pdflocal',pdflocal

    sess = requests.Session()
    sess.headers['Connection'] = 'close' 

    from azure.storage.blob import BlockBlobService
    blob_service = BlockBlobService(account_name=account_name, account_key=account_key,protocol='http', request_session=sess)
    #retrieve pdfname from blob and write it as pdflocal locally
    blob_service.get_blob_to_path(container_name, pdfname, pdflocal)

    ##### done with blob #### 


#     #calling function pdf2txt defined above
#     print "calling function pdf2txt \n"

#     output_tsv =  pdfname + '.txt'  
#     textfile = './' + output_tsv #local textfile
#     #print 'local textfile = ',textfile

#     pdf2txt(["firstarg","-o",textfile,pdfname])
#     print 'pdf converted to text and saved as text in',textfile
# #     print os.path.getsize(textfile)

#     #print 'output_tsv =',output_tsv  #to write to blob
#     #write the pdf back to blob
#     with open(textfile) as openfile:
#         for line in openfile:
#             line.strip().split('/n')
#             if 'HIV' in line:
#                 print line
#                 blob_service.create_blob_from_path(container_name, output_tsv,textfile)

else:
    print 'No download option provided'
    #return(dataframe1)  #works but is it better than sys.exit
    sys.exit("Enter inputs correctly. Provide atleast a url for pdf download (on line 5)")

doption is either 'fromblob' or from 'fromurl'
download option form user =  fromblob
user BYOB
calling function pdf2txt 

pdf converted to text and saved as text in ./report1.pdf.txt
K.  Have you ever tested for HIV ? If so, what was the result ? 



In [20]:
from azure.storage.blob import BlockBlobService

blob_service = BlockBlobService(account_name=account_name, account_key=account_key)

blobs = []
marker = None
while True:
    batch = blob_service.list_blobs('pdfcontainer2', marker=marker)
    blobs.extend(batch)
    if not batch.next_marker:
        break
    marker = batch.next_marker
for blob in blobs:
    print(blob.name)
    pdfname= blob.name
    print 'user BYOB'
    pdflocal = './' + pdfname #local file
    #print 'pdflocal',pdflocal

    sess = requests.Session()
    sess.headers['Connection'] = 'close' 
    
    #retrieve pdfname from blob and write it as pdflocal locally
    blob_service.get_blob_to_path(container_name, pdfname, pdflocal)
    print "calling function pdf2txt \n"

    output_tsv =  pdfname + '.txt'  
    textfile = './' + output_tsv #local textfile
    print 'local textfile = ',textfile

    pdf2txt(["firstarg","-o",textfile,pdfname])
    print 'pdf converted to text and saved as text in',textfile
#     print os.path.getsize(textfile)

    #print 'output_tsv =',output_tsv  #to write to blob
    #write the pdf back to blob
    with open(textfile) as openfile:
        for line in openfile:
            line.strip().split('/n')
            if 'Medical' in line:
                print line
#                 blob_service.create_blob_from_path(container_name, output_tsv,textfile)

report1.pdf
user BYOB
calling function pdf2txt 

local textfile =  ./report1.pdf.txt
pdf converted to text and saved as text in ./report1.pdf.txt
Is there any other item in your Medical History which you have not already mentioned ? 

Medical Examiner. If, however, the Bank is satisfied on the basis of the evidence produced before it, of 

the possibility of error of Judgement in the decision of the Medical Examiner, it is open to the Bank to 

allow an appeal to the Medical Board, which will be constituted by the Bank. Such evidence should 

decision of the Medical Examiner is advised to him/her. If the setting up of Medical Board is decided by 

medically fit by the Medical Board, this deposit would be refunded to the candidate. It will otherwise be 

forfeited. The Report of the Medical Board is final and will not be subjected to review by any other 

report2.pdf
user BYOB
calling function pdf2txt 

local textfile =  ./report2.pdf.txt
pdf converted to text and saved as text in ./rep